# Quantitative analysis

## Import libraries

In [1]:
import pandas as pd
import math
import scipy
import numpy as np
from skimage.metrics import structural_similarity as ssim
from scipy.stats import entropy
from numpy.linalg import norm
from tqdm import tqdm

## Metadata

In [2]:
# Background size
BG_SIZE_X=1920
BG_SIZE_Y=1080

## Functions

In [3]:
def splitFixationsToBuckets(df, bin_size):
    buckets = np.zeros((bin_size['x'], bin_size['y']))

    for _, fixation in df.iterrows():
        x = max(min(math.trunc((fixation['Fixation_X'] / BG_SIZE_X) * bin_size['x']), bin_size['x'] - 1), 0)
        y = max(min(math.trunc((fixation['Fixation_Y'] / BG_SIZE_Y) * bin_size['y']), bin_size['y'] - 1), 0),
        buckets[x][y] += 1
    buckets = buckets.flatten()
    totalSum = buckets.sum()
    
    return [x / totalSum for x in buckets]

def upscale_buckets(buckets, bin_size):
    expanded_width = np.repeat(np.reshape(buckets, (bin_size['x'], bin_size['y'])), BG_SIZE_X / bin_size['x'], axis=0)
    return np.repeat(expanded_width, BG_SIZE_Y / bin_size['y'], axis=1)

def JSD(P, Q):
    _P = P / norm(P, ord=1)
    _Q = Q / norm(Q, ord=1)
    _M = 0.5 * (_P + _Q)
    
    nonzero_indices = (_P > 0) & (_Q > 0)
    _P = _P[nonzero_indices]
    _Q = _Q[nonzero_indices]
    _M = _M[nonzero_indices]
    
    return 0.5 * (entropy(_P, _M) + entropy(_Q, _M))


def calculate_metrics(bin_size):
    metrics = []
    
    for stimulus in df_metadata['Stimulus'].unique():
        buckets_tobii = splitFixationsToBuckets(df_tobii.loc[(df_tobii['Stimulus'] == stimulus)], bin_size)
        buckets_rage_net = splitFixationsToBuckets(df_rage_net.loc[(df_rage_net['Stimulus'] == stimulus)], bin_size)

        tobii_ssim_map = upscale_buckets(buckets_tobii, bin_size)
        rage_net_ssim_map = upscale_buckets(buckets_rage_net, bin_size)
        ssim_score, _ = ssim(tobii_ssim_map, rage_net_ssim_map, full=True)
        
        metrics.append([
            stimulus,
            scipy.stats.pearsonr(buckets_tobii, buckets_rage_net)[0],
            scipy.stats.spearmanr(buckets_tobii, buckets_rage_net)[0],
            scipy.stats.kendalltau(buckets_tobii, buckets_rage_net)[0],
            ssim_score,
            JSD(buckets_rage_net, buckets_tobii),
        ])

    return pd.DataFrame(metrics, columns=['Stimulus', 'Pearson', 'Spearman', 'KendallTau', 'SSIM', 'JSD'])

def gaussian_mask(sizex, sizey, sigma=33, center=None, fix=1):
    x = np.arange(0, sizex, 1, float)
    y = np.arange(0, sizey, 1, float)
    x, y = np.meshgrid(x, y)

    if center is None:
        x0 = sizex // 2
        y0 = sizey // 2
    else:
        if np.isnan(center[0]) == False and np.isnan(center[1]) == False:
            x0 = center[0]
            y0 = center[1]
        else:
            return np.zeros((sizey, sizex))

    return fix * np.exp(-4 * np.log(2) * ((x - x0) ** 2 + (y - y0) ** 2) / sigma ** 2)

def build_heatmap(fixations, sigma=96):
    H, W = (BG_SIZE_Y, BG_SIZE_X)
    heatmap = np.zeros((H, W), np.float32)
    for index in tqdm(range(fixations.shape[0])):
        heatmap += gaussian_mask(W, H, sigma, (fixations[index, 0], fixations[index, 1]),
                                 fixations[index, 2])

    return heatmap / np.amax(heatmap)

## Load dataset

In [4]:
df_tobii = pd.read_csv('../../Datasets/Study_1/Fixations_tobii-tx-300.tsv', sep='\t')
df_rage_net = pd.read_csv('../../Datasets/Study_1/Fixations_rage-net.tsv', sep='\t')
df_metadata = pd.read_csv('../../Datasets/Study_1/Metadata.tsv', sep='\t')

### Calculate metrics for different grid sizes (X, Y)

In [5]:
calculate_metrics({'x': 4, 'y': 3}).head(6) 

,Stimulus,Pearson,Spearman,KendallTau,SSIM,JSD
0,UXtweak,0.161696,0.077058,0.076338,0.680795,0.077364
1,NetworkingAcademy,0.425906,0.253527,0.201605,0.633069,0.061984
2,SportsHere,0.492778,0.472855,0.320620,0.699263,0.074543
3,CodeHub,0.288783,0.702989,0.465130,0.731474,0.155706
4,RunTrack,0.483125,0.196839,0.201605,0.649522,0.145170
5,Listenify,0.812072,0.900000,0.769231,0.815623,0.040821


In [6]:
calculate_metrics({'x': 5, 'y': 3}).head(6) 

,Stimulus,Pearson,Spearman,KendallTau,SSIM,JSD
0,UXtweak,0.052439,-0.051030,-0.038648,0.692222,0.095088
1,NetworkingAcademy,0.492046,0.357735,0.278693,0.655732,0.059010
2,SportsHere,0.628240,0.441658,0.312288,0.683802,0.062228
3,CodeHub,0.374721,0.698374,0.552771,0.700624,0.153827
4,RunTrack,0.558640,0.304505,0.247537,0.679388,0.106170
5,Listenify,0.828167,0.658251,0.547270,0.764205,0.042396


In [7]:
calculate_metrics({'x': 5, 'y': 4}).head(6) 

,Stimulus,Pearson,Spearman,KendallTau,SSIM,JSD
0,UXtweak,0.099574,-0.037822,-0.038148,0.712101,0.060204
1,NetworkingAcademy,0.384580,0.240908,0.152698,0.613023,0.113292
2,SportsHere,0.289670,0.389974,0.254206,0.672600,0.113911
3,CodeHub,0.447936,0.461044,0.410538,0.722400,0.134948
4,RunTrack,0.343437,0.266869,0.186813,0.642963,0.168748
5,Listenify,0.390101,0.411183,0.312971,0.634317,0.125045


In [8]:
calculate_metrics({'x': 8, 'y': 4}).head(6) 

,Stimulus,Pearson,Spearman,KendallTau,SSIM,JSD
0,UXtweak,0.201622,0.070336,0.050580,0.718695,0.084991
1,NetworkingAcademy,0.383115,0.280529,0.198382,0.657878,0.131093
2,SportsHere,0.318251,0.421227,0.314635,0.703354,0.110261
3,CodeHub,0.493493,0.425406,0.394743,0.769223,0.096368
4,RunTrack,0.392058,0.319597,0.258943,0.714760,0.140217
5,Listenify,0.338442,0.446161,0.356040,0.713316,0.120029


In [9]:
calculate_metrics({'x': 10, 'y': 8}).head(6) 

,Stimulus,Pearson,Spearman,KendallTau,SSIM,JSD
0,UXtweak,0.012291,-0.003955,-0.006162,0.753143,0.135219
1,NetworkingAcademy,0.297251,0.182393,0.150677,0.772598,0.116752
2,SportsHere,0.099819,0.221766,0.175024,0.769340,0.148955
3,CodeHub,0.348487,0.404965,0.340549,0.813808,0.128865
4,RunTrack,0.410084,0.228463,0.183911,0.790957,0.155524
5,Listenify,0.181316,0.244244,0.208600,0.786054,0.162844


In [10]:
calculate_metrics({'x': 13, 'y': 10}).head(6) 

,Stimulus,Pearson,Spearman,KendallTau,SSIM,JSD
0,UXtweak,-0.012688,0.074278,0.057021,0.836715,0.165828
1,NetworkingAcademy,0.344286,0.214309,0.175479,0.837467,0.137845
2,SportsHere,0.241685,0.290482,0.235349,0.859266,0.107779
3,CodeHub,0.311190,0.447749,0.389350,0.863529,0.102749
4,RunTrack,0.321784,0.220129,0.183735,0.855912,0.107190
5,Listenify,0.120766,0.332131,0.283881,0.839107,0.157442


In [11]:
calculate_metrics({'x': 16, 'y': 9}).head(6) 

,Stimulus,Pearson,Spearman,KendallTau,SSIM,JSD
0,UXtweak,-0.064537,0.006864,0.002868,0.841169,0.118443
1,NetworkingAcademy,0.294283,0.247182,0.206045,0.846079,0.165396
2,SportsHere,0.225625,0.342934,0.282295,0.866765,0.128650
3,CodeHub,0.334201,0.456548,0.395897,0.864891,0.131112
4,RunTrack,0.420593,0.214537,0.179695,0.864797,0.115017
5,Listenify,0.193255,0.277570,0.238742,0.854794,0.155364


### Calculate metrics for gaussian heatmap

In [12]:
metrics = []

for stimulus in df_metadata['Stimulus'].unique():
    df_subset_tobii = df_tobii.loc[(df_tobii['Stimulus'] == stimulus)]
    df_subset_rage = df_rage_net.loc[(df_rage_net['Stimulus'] == stimulus)]

    heatmap_tobii = build_heatmap(df_subset_tobii[['Fixation_X', 'Fixation_Y', 'Duration']].to_numpy())
    heatmap_tobii_flat = heatmap_tobii.flatten()
    
    heatmap_rage = build_heatmap(df_subset_rage[['Fixation_X', 'Fixation_Y', 'Duration']].to_numpy())
    heatmap_rage_flat = heatmap_rage.flatten()
    
    ssim_score, _ = ssim(heatmap_tobii, heatmap_rage, full=True)
    
    metrics.append([
        stimulus,
        scipy.stats.pearsonr(heatmap_tobii_flat, heatmap_rage_flat)[0],
        scipy.stats.spearmanr(heatmap_tobii_flat, heatmap_rage_flat)[0],
        scipy.stats.kendalltau(heatmap_tobii_flat, heatmap_rage_flat)[0],
        ssim_score,
        JSD(heatmap_tobii_flat, heatmap_rage_flat),
    ])

pd.DataFrame(metrics, columns=['Stimulus', 'Pearson', 'Spearman', 'KendallTau', 'SSIM', 'JSD']).head(6)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 329/329 [00:29<00:00, 11.25it/s]


,Stimulus,Pearson,Spearman,KendallTau,SSIM,JSD
0,UXtweak,-0.010840,0.075810,0.050763,0.454955,0.308555
1,NetworkingAcademy,0.422080,0.398727,0.274175,0.591364,0.286266
2,SportsHere,0.249448,0.472744,0.331734,0.530739,0.262185
3,CodeHub,0.299382,0.639624,0.480967,0.695121,0.277856
4,RunTrack,0.431987,0.283109,0.196562,0.624947,0.279254
5,Listenify,0.152508,0.490645,0.344770,0.624319,0.358436
